In [1]:
import tensorflow as tf
sess = tf.InteractiveSession()

In [2]:
def abc(input, scope=None, reuse=None):
    with tf.variable_scope(scope, 'chg', reuse=reuse) as scope:
        x = tf.get_variable("abc", [input.get_shape()[0], input.get_shape()[1]])
        scope.reuse_variables() #this will not work for anther abc() no use!
        return x

In [3]:
x = tf.constant([[1,2, 3], [3,4, 5]], dtype=tf.float32)
y = tf.constant([[1,2,3,4], [5,6,7,8]], dtype=tf.float32)

sess.run(tf.global_variables_initializer())

In [4]:
x1 = abc(x, 'ofo')
#print x1.eval()
print x1

<tf.Variable 'ofo/abc:0' shape=(2, 3) dtype=float32_ref>


In [5]:
x1 = abc(x, 'ofo')
#print x1.eval()
print x1

ValueError: Variable ofo/abc already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "<ipython-input-2-af6f3a9cc28f>", line 3, in abc
    x = tf.get_variable("abc", [input.get_shape()[0], input.get_shape()[1]])
  File "<ipython-input-4-c2c0128fefae>", line 1, in <module>
    x1 = abc(x, 'ofo')
  File "/usr/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
x1 = abc(x, 'ofo', reuse=True)
#print x1.eval()
print x1

In [ ]:
def test(scope=None):
    with tf.variable_scope(scope, 'test'):
        x = tf.get_variable('abcd', [1,])
        print x
    return x + 1

In [ ]:
def run_test():
    with tf.variable_scope('new') as scope:
        u = test(scope)
        print u
        scope.reuse_variables()
        v = test(scope)
        print v
        return u, v

In [ ]:
u, v = run_test() #not work! still need reuse..

In [ ]:
for item in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES):
    print item

In [ ]:
def prepare_attention(attention_states,num_units, reuse=False):
  with tf.variable_scope("attention_keys", reuse=reuse) as scope:
    attention_keys = tf.contrib.layers.linear(
        attention_states, num_units, biases_initializer=None, scope=scope)
    return attention_keys

In [ ]:
a = prepare_attention(x, 10)
print a

In [ ]:
tf.get_variable_scope().reuse_variables()
b = prepare_attention(y, 10)
print b

In [ ]:
#below is how to escape variable scope

In [6]:
def create_construct_fn(name, reuse=None):
  with tf.variable_scope(name, reuse=reuse) as scope:
    def construct_fn():
      x = tf.get_variable('abcde', [1,])
      print(x)
      scope.reuse_variables()
      return x

    return construct_fn

In [7]:
fn = create_construct_fn('123')

In [8]:
fn()

<tf.Variable 'abcde:0' shape=(1,) dtype=float32_ref>


<tf.Variable 'abcde:0' shape=(1,) dtype=float32_ref>

In [9]:
def normal_fn(name, reuse=None):
    with tf.variable_scope(name, reuse=reuse) as scope:
        x = tf.get_variable('normal', [1,])
        print x

In [10]:
with tf.variable_scope('xxx'):
    x = tf.get_variable('uvw', [1,])
    print(x)
    fn()
    normal_fn('hahaha')

<tf.Variable 'xxx/uvw:0' shape=(1,) dtype=float32_ref>
<tf.Variable 'xxx/abcde:0' shape=(1,) dtype=float32_ref>
<tf.Variable 'xxx/hahaha/normal:0' shape=(1,) dtype=float32_ref>


In [11]:
def create_escape_construct_fn(name, reuse=None):
  with tf.variable_scope(name, reuse=reuse) as scope:
    def construct_fn(name2=''):
      with tf.variable_scope(scope) as scope_:
          x = tf.get_variable('escape', [1,])
          print(x)
          with tf.variable_scope(name2):
                y= tf.get_variable('alasky', [1,])
                print(y)
          scope.reuse_variables() #without this In[16] will fail can not visit again
      return x

    return construct_fn

In [12]:
escape_fn = create_escape_construct_fn('try_escape')
escape_fn2 = create_escape_construct_fn('')

In [13]:
with tf.variable_scope('yyy'):
    x = tf.get_variable('uvw', [1,])
    print(x)
    fn()
    normal_fn('hahaha')
    escape_fn()
    escape_fn2()

<tf.Variable 'yyy/uvw:0' shape=(1,) dtype=float32_ref>
<tf.Variable 'yyy/abcde:0' shape=(1,) dtype=float32_ref>
<tf.Variable 'yyy/hahaha/normal:0' shape=(1,) dtype=float32_ref>
<tf.Variable 'try_escape/escape:0' shape=(1,) dtype=float32_ref>
<tf.Variable 'try_escape//alasky:0' shape=(1,) dtype=float32_ref>
<tf.Variable 'escape:0' shape=(1,) dtype=float32_ref>
<tf.Variable 'alasky:0' shape=(1,) dtype=float32_ref>


In [14]:
escape_fn3 = create_escape_construct_fn('XXX')

In [15]:
escape_fn3()
x = tf.get_variable('uvw', [1,])
print x

<tf.Variable 'XXX/escape:0' shape=(1,) dtype=float32_ref>
<tf.Variable 'XXX//alasky:0' shape=(1,) dtype=float32_ref>
<tf.Variable 'uvw:0' shape=(1,) dtype=float32_ref>


In [16]:
escape_fn()

<tf.Variable 'try_escape/escape:0' shape=(1,) dtype=float32_ref>
<tf.Variable 'try_escape//alasky:0' shape=(1,) dtype=float32_ref>


<tf.Variable 'try_escape/escape:0' shape=(1,) dtype=float32_ref>

In [17]:
escape_fn2()

<tf.Variable 'escape:0' shape=(1,) dtype=float32_ref>
<tf.Variable 'alasky:0' shape=(1,) dtype=float32_ref>


<tf.Variable 'escape:0' shape=(1,) dtype=float32_ref>

In [18]:
escape_fn2('dsfddsfdsf')

<tf.Variable 'escape:0' shape=(1,) dtype=float32_ref>


ValueError: Variable dsfddsfdsf/alasky does not exist, or was not created with tf.get_variable(). Did you mean to set reuse=None in VarScope?

In [19]:
escape_fn3 = create_escape_construct_fn('XXX')

In [20]:
escape_fn3()

ValueError: Variable XXX/escape already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "<ipython-input-11-099841162517>", line 5, in construct_fn
    x = tf.get_variable('escape', [1,])
  File "<ipython-input-15-7da7ada1d608>", line 1, in <module>
    escape_fn3()
  File "/usr/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)


In [21]:
escape_fn3 = create_escape_construct_fn('XXX', reuse=True)

In [22]:
escape_fn3()

<tf.Variable 'XXX/escape:0' shape=(1,) dtype=float32_ref>
<tf.Variable 'XXX//alasky:0' shape=(1,) dtype=float32_ref>


<tf.Variable 'XXX/escape:0' shape=(1,) dtype=float32_ref>

In [23]:
with tf.variable_scope('XXX'):
    x = tf.get_variable('alasky', [1,])
    print(x)

<tf.Variable 'XXX/alasky:0' shape=(1,) dtype=float32_ref>


In [24]:
with tf.variable_scope('XXX'):
    x = tf.get_variable('alasky', [1,])
    print(x)

ValueError: Variable XXX/alasky already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "<ipython-input-23-47f82ac909cb>", line 2, in <module>
    x = tf.get_variable('alasky', [1,])
  File "/usr/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/usr/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2821, in run_ast_nodes
    if self.run_code(code, result):


In [25]:
escape_fn4 = create_escape_construct_fn('', reuse=True)

In [26]:
escape_fn4()

<tf.Variable 'escape:0' shape=(1,) dtype=float32_ref>
<tf.Variable 'alasky:0' shape=(1,) dtype=float32_ref>


<tf.Variable 'escape:0' shape=(1,) dtype=float32_ref>

In [28]:
with tf.variable_scope('XXX'):
    x = tf.get_variable('alasky', [1,])
    print(x)

ValueError: Variable XXX/alasky already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "<ipython-input-23-47f82ac909cb>", line 2, in <module>
    x = tf.get_variable('alasky', [1,])
  File "/usr/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/usr/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2821, in run_ast_nodes
    if self.run_code(code, result):


In [29]:
escape_fn3()

<tf.Variable 'XXX/escape:0' shape=(1,) dtype=float32_ref>
<tf.Variable 'XXX//alasky:0' shape=(1,) dtype=float32_ref>


<tf.Variable 'XXX/escape:0' shape=(1,) dtype=float32_ref>

In [30]:
with tf.variable_scope('XXX'):
    x = tf.get_variable('alasky', [1,])
    print(x)

ValueError: Variable XXX/alasky already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "<ipython-input-23-47f82ac909cb>", line 2, in <module>
    x = tf.get_variable('alasky', [1,])
  File "/usr/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/usr/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2821, in run_ast_nodes
    if self.run_code(code, result):


In [31]:
escape_fn3()

<tf.Variable 'XXX/escape:0' shape=(1,) dtype=float32_ref>
<tf.Variable 'XXX//alasky:0' shape=(1,) dtype=float32_ref>


<tf.Variable 'XXX/escape:0' shape=(1,) dtype=float32_ref>

In [32]:
with tf.variable_scope('XXX'):
    x = tf.get_variable('escape', [1,])
    print(x)
    
#so XXX only be defualt reused in escape_fn3() function...

ValueError: Variable XXX/escape already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "<ipython-input-11-099841162517>", line 5, in construct_fn
    x = tf.get_variable('escape', [1,])
  File "<ipython-input-15-7da7ada1d608>", line 1, in <module>
    escape_fn3()
  File "/usr/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)


In [35]:
with tf.variable_scope('ddd'):
    x = tf.get_variable('alasky', [1,])
    print(x)
    with tf.variable_scope(tf.get_variable_scope()):
        y = tf.get_variable('husky', [1,])
        print(y)

<tf.Variable 'ddd/alasky:0' shape=(1,) dtype=float32_ref>
<tf.Variable 'ddd/husky:0' shape=(1,) dtype=float32_ref>
